In [ ]:
import numpy as np
from random import randint
import copy

In [ ]:
def ler_entrada(filename):
    arquivo = open(filename,"r")
    linhas = arquivo.read()
    arquivo.close()
    vetor = list(map(int, linhas.split()))
    capacidade = vetor[0]
    vetor = vetor[1:]
    return capacidade, vetor

def calFitness(s,p,c, beta):
    # s => solucao (bool)
    # p => pesos (int)
    # c => capcidade maxima
    # s e p tem a mesma dimensão
    acumm = 0
    
    for i in range(len(p)):
        acumm += p[i] * s[i]
    
    #print(acumm)
    return acumm + beta * max(0, acumm - c)


def idxZero(s, n):
    # retorna o indice de um item que não esta na mochila
    idxRand = randint(0,n-1)
    for j in range(n):
        if s[(j+idxRand)%n] == 0:
            return ((j+idxRand)%n)
    return -1
            
def idxOne(s, n):
    # retorna o indice de um item que esta na mochila
    idxRand = randint(0,n-1)
    for j in range(n):
        if s[(j+idxRand)%n] == 1:
            return ((j+idxRand)%n)
    return -1


def VND(s, p, c, beta, qtdItr):
    # s => solucao
    # p => pesos
    # c => capcidade maxima
    # beta => valor da fitness
    # qtdItr => quantidade de iteracoes do for
    bSol = s
    cFit = calFitness(s,p,c,beta)
    bFit = cFit
    n = len(s)
    
    # verifica se eh uma solucao otima
    if c - cFit == 0:
        return bSol
    
    var_idxZero = -1
    var_idxOne = -1
    
    # numero de tentativas em busca de uma solucao otima
    for i in range(qtdItr):
        var_idxZero = idxZero(s, n)
        var_idxOne = idxOne(s, n)
        
        #print (var_idxZero, var_idxOne)
        
        # se ainda ha espaco
        if (c - cFit) > 0: 
            # vejo se a troca não transborda a mochila
            if p[var_idxZero] - p[var_idxOne] <= c - cFit: 
                s[var_idxOne] = 0
                s[var_idxZero] = 1  
        # se nao ha espaco, vamos reduzir o peso
        elif p[var_idxZero] - p[var_idxOne] < 0:
            s[var_idxOne] = 0
            s[var_idxZero] = 1
        # calcula o novo fitness
        cFit = calFitness(s,p,c,beta)
        # verifica se eh uma solucao otima
        if c - cFit == 0:
            return bSol
        #  verifica se a solucao atual ...
        #...esta mais proxima que a melhor encontrada ate o momento
        # e atualiza a nova melhor solucao
        if abs(c - cFit) < abs(c - bFit):
            bFit = cFit
            bSol = s
    
    if c - bFit >= 0:
        return bSol
    else:
        return ([0]*n)
    
def VNDR(s, p, c, beta, seed, qtdItr, debug = 0):
    # s => solucao
    # p => pesos
    # c => capcidade maxima
    # beta => valor da fitness
    # qtdItr => quantidade de iteracoes do for
    # seed => porcentagem de chance de inserir 
    #         um novo elemento na mochila [0,100]
    bSol = s
    cFit = calFitness(s,p,c,beta)
    bFit = cFit
    n = len(s)
    percent = 100/seed;
    countItr = 0;
    
    # verifica se eh uma solucao otima
    if c - cFit == 0:
        if debug:
            print 'return 1: ', cFit
        return countItr,bSol
    
    var_idxZero = -1
    var_idxOne = -1
    
    # numero de tentativas em busca de uma solucao otima
    for i in range(qtdItr):
        countItr += 1
        var_idxZero = idxZero(s, n)
        var_idxOne = idxOne(s, n)
        
        #free_space = c - cFit
        
        # se ainda ha espaco
        if (c - cFit) > 0:
            # chance de acrescentar um novo elemento na mochila
            if randint(1,percent) == 1:
                s[var_idxZero] = 1
                
            # vejo se a troca não transborda a mochila
            elif p[var_idxZero] - p[var_idxOne] <= c - cFit: 
                s[var_idxOne] = 0
                s[var_idxZero] = 1  
                
        # se nao ha espaco, vamos reduzir o peso
        elif randint(1,percent) == 1:
            s[var_idxOne] = 0
        elif p[var_idxZero] - p[var_idxOne] < 0:
            s[var_idxOne] = 0
            s[var_idxZero] = 1
        # calcula o novo fitness
        cFit = calFitness(s,p,c,beta)
        #free_space = c - cFit
        
        # verifica se eh uma solucao otima
        if c - cFit == 0:
            if debug:
                print 'return 1: ', cFit
            return countItr,s
        
        #  verifica se a solucao atual ...
        #...esta mais proxima que a melhor encontrada ate o momento
        # e atualiza a nova melhor solucao
        if abs(c - cFit) < abs(c - bFit):
            bSol = copy.deepcopy (s)
            bFit = calFitness (bSol, p, c, beta)
            
    if debug:
        print (c)
        print (bFit)
        print c - bFit
    
    if c - bFit >= 0:
        if debug:
            print('return 2')
        return countItr,bSol
    else:
        if debug:
            print('return 3')
        return countItr,([0]*n)

In [ ]:
c,p = ler_entrada('p7.txt')
#print 'capacidade: ', c

s =  [0]*len(p)

beta = c*c
seed = 45
qtdItr = 10000

#bSol = VND(s, p, c, beta, 100)


qtdItrProcessada, bSolR = VNDR(s[:], p[:], c, beta, seed, qtdItr)

print 'Espaco sobrando: ',c - calFitness(bSolR,p,c,beta), '; Interacoes: ', qtdItrProcessada


In [ ]:
c,p = ler_entrada('p3.txt')
#print 'capacidade: ', c

s =  [0]*len(p)

beta = c*c
qtdItr = 10000


for seed in [10,20,30,40,50,60,70,80,90,100]:
    mean = 0.0

    for j in xrange (10):
        i = 0
        flag = 0

        while (i < 1000):
            i += 1
            qtdItrProcessada, bSolR = VNDR(s[:], p[:], c, beta, seed, qtdItr)
            if c - calFitness(bSolR,p,c,beta) != 0:
                flag += 1
        mean += flag
        #print j, ': ', flag

    mean = mean / 10.0

    print seed, '%; 0 media: ', mean